In [1]:
import utils
import threading
from datetime import datetime as dt
from playsound import playsound
from configs import API_KEY,BASE_URL,AMOUNT,MIN_PERCENT_REQUIRED
from biscoint import get_rate_limit,get_balance,get_offer,confirm_offer
import time

In [64]:
#Inicial configs
rate_limit = get_rate_limit()
sleep_time_offers = ((rate_limit["windowMs"] / rate_limit["maxRequests"]) / 1000) * 1.5
no_rush_percent = -0.2 # If the spread is lower than this, wait normal time
initial_balance = get_balance()
last_balance = initial_balance

In [5]:
%%time
buy = get_offer('buy','0.0001',False)
sell = get_offer('sell','0.0001',False)
print(buy)
print(sell)

{'offerId': 'WLfXorKZtmFWdARLP', 'base': 'BTC', 'quote': 'BRL', 'op': 'buy', 'isQuote': False, 'baseAmount': '0.00010000', 'quoteAmount': '31.14', 'efPrice': '311400.00', 'createdAt': '2021-04-20T00:30:48.034Z', 'expiresAt': '2021-04-20T00:31:03.034Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}
{'offerId': 'uBmJkTrMEjW6cSrbn', 'base': 'BTC', 'quote': 'BRL', 'op': 'sell', 'isQuote': False, 'baseAmount': '0.00010000', 'quoteAmount': '30.97', 'efPrice': '309700.00', 'createdAt': '2021-04-20T00:30:49.583Z', 'expiresAt': '2021-04-20T00:31:04.583Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}
Wall time: 3.19 s


In [6]:
def async_offer(op:str,amount:str,isQuote:bool):
    try:
        response = get_offer(op,amount,isQuote)
        request_orders[op] = response
    except Exception as e:
        print("Erro ao rodar thread",e)

In [66]:
while True:
    start_time = dt.now()
    try:
        request_orders = {'buy':None,'sell':None}

        # thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
        # thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
        # thread_buy.start()
        # time.sleep(0.1)
        # thread_sell.start()
        # thread_buy.join()
        # thread_sell.join()
        request_orders['buy'] = get_offer('buy',AMOUNT,False)
        request_orders['sell'] = get_offer('sell',AMOUNT,False)

        buy_price = request_orders['buy']['efPrice']
        sell_price = request_orders['sell']['efPrice']
        percent = utils.percent(buy_price, sell_price)

        if percent >= MIN_PERCENT_REQUIRED:
            print(f"Arbitrage oportunity: buy:{buy_price}   sell:{sell_price} ->  {percent}%")
            playsound('beep.wav')
            executed_buy = confirm_offer(buy['offerId'])
            executed_sell = confirm_offer(sell['offerId'])

        print(percent)

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        print(f"Took {seconds_elapsed} seconds")
        print("Wait",sleep_time_offers-percent)
        if percent <= no_rush_percent:
            time.sleep(sleep_time_offers)
        else:
            time.sleep(0.5)
        
    except Exception as e:
        print(e)



('Erro ao buscar oferta', {'code': 4003, 'error': 'Forbidden', 'details': 'Invalid authorization headers'})
-0.519
Took 1.404456 seconds
Wait 3.519
-0.411
Took 2.001726 seconds
Wait 3.411
-0.394
Took 0.989271 seconds
Wait 3.394
-0.412
Took 1.603168 seconds
Wait 3.412
-0.286
Took 1.173942 seconds
Wait 3.286
-0.819
Took 2.339609 seconds
Wait 3.819
-0.766
Took 2.116955 seconds
Wait 3.766


KeyboardInterrupt: 

In [63]:
percent = -1
sleep_time_offers * percent

-2.0